## License 

Copyright 2021-2023 Patrick Hall (jphall@gwu.edu)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

*DISCLAIMER*: This notebook is not legal or compliance advice.

# Model Evaluation Notebook

#### Imports and inits

In [1]:
import os              # for directory and file manipulation
import numpy as np     # for basic array manipulation
import pandas as pd    # for dataframe manipulation
import datetime        # for timestamp

# for model eval
from sklearn.metrics import accuracy_score, f1_score, log_loss, mean_squared_error, roc_auc_score

# global constants 
ROUND = 3              # generally, insane precision is not needed 
SEED = 12345           # seed for better reproducibility

# set global random seed for better reproducibility
np.random.seed(SEED)

#### Set basic metadata

In [2]:
y_name = 'high_priced'
scores_dir = 'data/scores'

#### Read in score files 

In [3]:
# init score frame with known test y values
scores_frame = pd.read_csv(scores_dir + os.sep +'key.csv', index_col='Unnamed: 0')

# create random folds in reproducible way
np.random.seed(SEED)
scores_frame['fold'] = np.random.choice(5, scores_frame.shape[0])

# read in each score file in the directory as a new column 
for file in sorted(os.listdir(scores_dir)):
    if file != 'key.csv' and file.endswith('.csv'):
        scores_frame[file[:-4]] = pd.read_csv(scores_dir + os.sep + file)['phat']

# sanity check 
scores_frame

,high_priced,fold,group1_ebm,group1_glm,group1_mxgb,group2_piml_EBM,group2_piml_XGB2,group2_piml_reludnn,group3_piml_EBM,group3_piml_GAM,...,group5_xgb2,group8_piml_ebm,group8_piml_gam,group8_piml_reludnn,group9_ebm,group9_glm,group9_mxgb,ph_ebm,ph_glm,ph_mxgb
0,0.0,2,0.080557,0.142090,0.059522,0.060440,0.058846,0.054570,0.079611,0.109994,...,0.077672,0.073114,0.066971,0.066971,0.080557,0.142090,0.086361,0.080557,0.142090,0.071625
1,0.0,1,0.026001,0.081674,0.036210,0.030870,0.030136,0.032199,0.030699,0.050429,...,0.035456,0.029787,0.034471,0.034471,0.026001,0.081674,0.033920,0.026001,0.081674,0.036750
2,1.0,4,0.194961,0.125823,0.180734,0.184039,0.185935,0.200142,0.185728,0.166834,...,0.198679,0.178312,0.183367,0.183367,0.194961,0.125823,0.183323,0.194961,0.125823,0.168644
3,0.0,1,0.028556,0.006973,0.027677,0.023568,0.024420,0.012645,0.025441,0.020917,...,0.019302,0.019792,0.014284,0.014284,0.028556,0.006973,0.030934,0.028556,0.006973,0.029126
4,1.0,2,0.208263,0.130426,0.177813,0.203001,0.203261,0.177770,0.197092,0.167265,...,0.196543,0.182424,0.177948,0.177948,0.208263,0.130426,0.178491,0.208263,0.130426,0.179502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19826,0.0,3,0.228342,0.160032,0.274767,0.228772,0.206548,0.243380,0.207749,0.214998,...,0.232284,0.201933,0.254153,0.254153,0.228342,0.160032,0.255826,0.228342,0.160032,0.261347
19827,0.0,1,0.253998,0.123836,0.182039,0.218112,0.199714,0.164194,0.237054,0.138583,...,0.236322,0.212543,0.167147,0.167147,0.253998,0.123836,0.176984,0.253998,0.123836,0.185477
19828,1.0,3,0.213364,0.169604,0.212740,0.212545,0.211904,0.227022,0.252599,0.232105,...,0.202498,0.212675,0.230183,0.230183,0.213364,0.169604,0.236894,0.213364,0.169604,0.224887
19829,0.0,1,0.002176,0.002538,0.001323,0.005655,0.012688,0.000296,0.003545,0.004071,...,0.015741,0.007463,0.001132,0.001132,0.002176,0.002538,0.001113,0.002176,0.002538,0.000985


#### Utility function for max. accuracy

In [4]:
def max_acc(y, phat, res=0.01): 

    """ Utility function for finding max. accuracy at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. accuracy, default 0.01.
        :return: Max. accuracy for model scores.
    
    """
    
    # init frame to store acc at different cutoffs
    acc_frame = pd.DataFrame(columns=['cut', 'acc'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find accuracy at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        acc = accuracy_score(temp_df.iloc[:, 0], temp_df['decision'])
        acc_frame = acc_frame.append({'cut': cut,
                                      'acc': acc},
                                     ignore_index=True)

    # find max accurcay across all cutoffs
    max_acc = acc_frame['acc'].max()
    
    # house keeping
    del acc_frame, temp_df
    
    return max_acc

####  Utility function for max. F1

In [5]:
def max_f1(y, phat, res=0.01): 
    
    """ Utility function for finding max. F1 at some cutoff. 
    
        :param y: Known y values.
        :param phat: Model scores.
        :param res: Resolution over which to search for max. F1, default 0.01.
        :return: Max. F1 for model scores.
    
    """
    
    # init frame to store f1 at different cutoffs
    f1_frame = pd.DataFrame(columns=['cut', 'f1'])
    
    # copy known y and score values into a temporary frame
    temp_df = pd.concat([y, phat], axis=1)
    
    # find f1 at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        f1 = f1_score(temp_df.iloc[:, 0], temp_df['decision'])
        f1_frame = f1_frame.append({'cut': cut,
                                    'f1': f1},
                                    ignore_index=True)
        
    # find max f1 across all cutoffs
    max_f1 = f1_frame['f1'].max()
    
     # house keeping
    del f1_frame, temp_df
    
    return max_f1

#### Rank all submitted scores 

In [6]:
eval_frame = pd.DataFrame() # init frame to hold score ranking
metric_list = ['acc', 'auc', 'f1', 'logloss', 'mse'] # metric to use for evaluation

# create eval frame row-by-row
for fold in sorted(scores_frame['fold'].unique()): # loop through folds 
    for metric_name in metric_list: # loop through metrics
        
        # init row dict to hold each rows values
        row_dict = {'fold': fold,
                    'metric': metric_name}
        
        # cache known y values for fold
        fold_y = scores_frame.loc[scores_frame['fold'] == fold, y_name]
        
        for col_name in scores_frame.columns[2:]:
            
            # cache fold scores
            fold_scores = scores_frame.loc[scores_frame['fold'] == fold, col_name]
            
            # calculate evaluation metric for fold
            # with reasonable precision 
            
            if metric_name == 'acc':
                row_dict[col_name] = np.round(max_acc(fold_y, fold_scores), ROUND)
                
            if metric_name == 'auc':
                row_dict[col_name] = np.round(roc_auc_score(fold_y, fold_scores), ROUND)
                
            if metric_name == 'f1':
                row_dict[col_name] = np.round(max_f1(fold_y, fold_scores), ROUND) 
                
            if metric_name == 'logloss':
                row_dict[col_name] = np.round(log_loss(fold_y, fold_scores), ROUND)
                
            if metric_name == 'mse':
                row_dict[col_name] = np.round(mean_squared_error(fold_y, fold_scores), ROUND)
        
        # append row values to eval_frame
        eval_frame = eval_frame.append(row_dict, ignore_index=True)

# init a temporary frame to hold rank information
rank_names = [name + '_rank' for name in sorted(eval_frame.columns) if name not in ['fold', 'metric']]
rank_frame = pd.DataFrame(columns=rank_names)        

# set columns to necessary order
eval_frame = eval_frame[['fold', 'metric'] + [name for name in sorted(eval_frame.columns) if name not in ['fold', 'metric']]]

# determine score ranks row-by-row
for i in range(0, eval_frame.shape[0]):
        
        # get ranks for row based on metric
        metric_name = eval_frame.loc[i, 'metric']
        if metric_name in ['logloss', 'mse']:
            ranks = eval_frame.iloc[i, 2:].rank().values
        else:
            ranks = eval_frame.iloc[i, 2:].rank(ascending=False).values
        
        # create single-row frame and append to rank_frame
        row_frame = pd.DataFrame(ranks.reshape(1, ranks.shape[0]), columns=rank_names)
        rank_frame = rank_frame.append(row_frame, ignore_index=True)
        
        # house keeping
        del row_frame

# merge ranks onto eval_frame
eval_frame = pd.concat([eval_frame, rank_frame], axis=1)

# house keeping
del rank_frame
        
eval_frame

Empty DataFrame
Columns: [group1_ebm_rank, group1_glm_rank, group1_mxgb_rank, group2_piml_EBM_rank, group2_piml_XGB2_rank, group2_piml_reludnn_rank, group3_piml_EBM_rank, group3_piml_GAM_rank, group3_piml_XGB_rank, group5_gaminet_rank, group5_reludnn_rank, group5_xgb2_rank, group8_piml_ebm_rank, group8_piml_gam_rank, group8_piml_reludnn_rank, group9_ebm_rank, group9_glm_rank, group9_mxgb_rank, ph_ebm_rank, ph_glm_rank, ph_mxgb_rank]
Index: []

[0 rows x 21 columns]
    group1_ebm_rank  group1_glm_rank  group1_mxgb_rank  group2_piml_EBM_rank  \
0               9.0             17.5               2.5                   9.0   
1               2.0             20.0              15.5                   6.5   
2               2.0             20.0              12.0                   7.5   
3               2.5             20.0              15.0                   7.0   
4               5.0             20.0              13.0                   5.0   
5              11.0             11.0              

,fold,metric,group1_ebm,group1_glm,group1_mxgb,group2_piml_EBM,group2_piml_XGB2,group2_piml_reludnn,group3_piml_EBM,group3_piml_GAM,...,group5_xgb2_rank,group8_piml_ebm_rank,group8_piml_gam_rank,group8_piml_reludnn_rank,group9_ebm_rank,group9_glm_rank,group9_mxgb_rank,ph_ebm_rank,ph_glm_rank,ph_mxgb_rank
0,0.0,acc,0.901,0.900,0.902,0.901,0.900,0.900,0.902,0.900,...,9.0,9.0,9.0,9.0,9.0,17.5,2.5,9.0,17.5,2.5
1,0.0,auc,0.840,0.775,0.814,0.836,0.837,0.822,0.837,0.805,...,6.5,8.5,11.0,11.0,2.0,20.0,17.0,2.0,20.0,15.5
2,0.0,f1,0.405,0.335,0.379,0.395,0.394,0.373,0.399,0.358,...,7.5,5.0,15.0,15.0,2.0,20.0,15.0,2.0,20.0,11.0
3,0.0,logloss,0.251,0.291,0.263,0.253,0.253,0.259,0.251,0.271,...,7.0,7.0,11.0,11.0,2.5,20.0,15.0,2.5,20.0,15.0
4,0.0,mse,0.077,0.084,0.078,0.077,0.077,0.078,0.077,0.081,...,5.0,5.0,13.0,13.0,5.0,20.0,13.0,5.0,20.0,13.0
5,1.0,acc,0.906,0.906,0.906,0.906,0.906,0.906,0.906,0.906,...,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0,11.0
6,1.0,auc,0.828,0.757,0.792,0.826,0.826,0.801,0.828,0.780,...,8.5,8.5,13.0,13.0,2.5,20.0,16.0,2.5,20.0,16.0
7,1.0,f1,0.368,0.302,0.337,0.369,0.365,0.341,0.369,0.320,...,6.5,8.0,14.0,14.0,4.0,20.0,12.0,4.0,20.0,16.5
8,1.0,logloss,0.246,0.281,0.264,0.247,0.248,0.257,0.247,0.269,...,8.0,8.0,11.5,11.5,2.0,20.0,16.5,2.0,20.0,15.0
9,1.0,mse,0.074,0.080,0.078,0.075,0.075,0.077,0.074,0.079,...,7.0,7.0,11.5,11.5,2.5,20.0,15.5,2.5,20.0,15.5


#### Save `eval_frame` as CSV

In [7]:
eval_frame.to_csv('model_eval_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S") + '.csv'), 
                  index=False)

#### Display simple ranked score list 

In [8]:
eval_frame[[name for name in eval_frame.columns if name.endswith('rank')]].mean().sort_values()

group3_piml_EBM_rank         4.30
group1_ebm_rank              5.34
ph_ebm_rank                  5.34
group9_ebm_rank              5.34
group2_piml_EBM_rank         5.36
group5_xgb2_rank             5.60
group2_piml_XGB2_rank        6.20
group3_piml_XGB_rank         6.94
group8_piml_ebm_rank         7.52
group2_piml_reludnn_rank    11.70
group8_piml_reludnn_rank    11.84
group8_piml_gam_rank        11.84
group5_reludnn_rank         12.00
group5_gaminet_rank         13.76
ph_mxgb_rank                14.30
group1_mxgb_rank            14.50
group9_mxgb_rank            14.72
group3_piml_GAM_rank        17.34
group9_glm_rank             19.02
group1_glm_rank             19.02
ph_glm_rank                 19.02
dtype: float64